<a href="https://colab.research.google.com/github/PyBeginner1/FakeNewsPrediction/blob/main/FakeNewsPredcition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer


In [6]:
from google.colab import files
uploaded= files.upload()

Saving train.csv to train.csv


In [7]:
news = pd.read_csv('/content/train.csv')

In [10]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [13]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [27]:
news.head()

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy Iranian woman jailed for fictio...


In [8]:
news.shape


(20800, 5)

Check for null


In [9]:
news.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

Replace missing values with empty string

In [15]:
news = news.fillna('')

In [16]:
news.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

Combine title & author 

In [18]:
news['content'] = news['author']+ ' ' +news['title']

In [76]:
news['content']

0        [darrel, lucu, hous, dem, aid, even, see, come...
1        [daniel, j, flynn, flynn, hillari, clinton, bi...
2            [consortiumnew, com, truth, might, get, fire]
3        [jessica, purkiss, civilian, kill, singl, us, ...
4        [howard, portnoy, iranian, woman, jail, fictio...
                               ...                        
20795    [jerom, hudson, rapper, trump, poster, child, ...
20796    [benjamin, hoffman, n, f, l, playoff, schedul,...
20797    [michael, j, de, la, merc, rachel, abram, maci...
20798    [alex, ansari, nato, russia, hold, parallel, e...
20799                      [david, swanson, keep, f, aliv]
Name: content, Length: 20800, dtype: object

Seperate the data & labels

In [25]:
X = news.drop(columns = ['label','id'], axis = 1)
Y =news['label']

In [28]:
print(X)
print(Y)

                                                   title  ...                                            content
0      House Dem Aide: We Didn’t Even See Comey’s Let...  ...  Darrell Lucus House Dem Aide: We Didn’t Even S...
1      FLYNN: Hillary Clinton, Big Woman on Campus - ...  ...  Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2                      Why the Truth Might Get You Fired  ...  Consortiumnews.com Why the Truth Might Get You...
3      15 Civilians Killed In Single US Airstrike Hav...  ...  Jessica Purkiss 15 Civilians Killed In Single ...
4      Iranian woman jailed for fictional unpublished...  ...  Howard Portnoy Iranian woman jailed for fictio...
...                                                  ...  ...                                                ...
20795  Rapper T.I.: Trump a ’Poster Child For White S...  ...  Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...  ...  Benjamin Hoffman N.F.L. Playoffs:

Stemming : Stemming is the process of reducing  words to their root forms such as mapping a group of words to the same stem even if the stem itself is not a valid word in the Language

In [72]:
port_stem = PorterStemmer()

In [90]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',str(content))
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content


In [91]:
news['content'] = news['content'].apply(stemming)

In [92]:
news['content']

0        darrel lucu hou dem aid even see comey letter ...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exerci b...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object

Seperating data & values


In [93]:
X = news['content'].values
Y= news['label'].values

In [94]:
print(X)
print(Y)

['darrel lucu hou dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exerci balkan'
 'david swanson keep f aliv']
[1 0 1 ... 0 1 1]


Convert **Textual data** to **Numerical data**

In [95]:
vector = TfidfVectorizer()
vector.fit(X)

X= vector.transform(X)



In [98]:
print(X)


  (0, 15545)	0.28485063562728646
  (0, 13342)	0.2565896679337957
  (0, 8818)	0.3635963806326075
  (0, 8542)	0.29212514087043684
  (0, 7608)	0.24785219520671603
  (0, 6928)	0.21874169089359144
  (0, 4916)	0.233316966909351
  (0, 3759)	0.2705332480845492
  (0, 3567)	0.3598939188262559
  (0, 2933)	0.2468450128533713
  (0, 2464)	0.3676519686797209
  (0, 263)	0.27010124977708766
  (1, 16646)	0.3025156488372128
  (1, 6742)	0.19152496072048605
  (1, 5439)	0.7186013955384664
  (1, 3535)	0.2653147533915268
  (1, 2790)	0.19208753385709676
  (1, 2206)	0.36915639258038363
  (1, 1877)	0.15614790568229528
  (1, 1481)	0.2957471154505952
  (2, 15470)	0.41544962664721613
  (2, 9528)	0.49351492943649944
  (2, 5901)	0.3474613386728292
  (2, 5325)	0.3866530551182615
  (2, 3072)	0.46097489583229645
  :	:
  (20797, 12992)	0.2483705036831893
  (20797, 12222)	0.27276402145717243
  (20797, 12017)	0.24790022252744132
  (20797, 10212)	0.0804189541935242
  (20797, 9496)	0.17463635692029988
  (20797, 9427)	0.29394

Training & Testing


In [104]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y,  random_state = 2 )

In [105]:
print(X.shape, X_train.shape, X_test.shape)

(20800, 16974) (16640, 16974) (4160, 16974)


Model =  Logistic Regression


In [106]:
model = LogisticRegression()

Training the model

In [107]:
model.fit(X_train, Y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [109]:
train_news_prediction = model.predict(X_train)

Accuracy Score of Training model

In [111]:
score = accuracy_score(Y_train,train_news_prediction)
print("Accuracy score of training model is" ,score)

Accuracy score of training model is 0.9866586538461538


Accuracy on test data

In [116]:
test_data_prediction = model.predict(X_test)
score_1  = accuracy_score(test_data_prediction, Y_test)
print("Accuracy of testing data is", score_1)

Accuracy of testing data is 0.9790865384615385


Buildidng a predictive System

In [120]:
input_data = X_test[0]

prediction = model.predict(input_data)
if prediction == 1:
  print('Fake News')
else:
  print("Its Real!!!")

Fake News


In [121]:
input_data1 = X_test[1]

prediction = model.predict(input_data1)
if prediction == 1:
  print('Fake News')
else:
  print("Its Real!!!")

Its Real!!!
